In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.models import resnet152
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import splitfolders

from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import os
from PIL import Image

In [19]:
# Путь к папке с изображениями
data_dir = 'D:\\online_school\\5thLesson\\data\\caltech101\\101_ObjectCategories'

# Список всех файлов изображений
all_images = []
labels = []

# Размер, к которому будут приводиться все изображения
target_size = (64, 64)

for category in os.listdir(data_dir):
    category_path = os.path.join(data_dir, category)
    if os.path.isdir(category_path):
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            # Открытие изображения и изменение его размера
            img = Image.open(img_path).convert('RGB')  # Преобразование в RGB
            img = img.resize(target_size)
            all_images.append(np.array(img))
            labels.append(category)

# Преобразование в массивы numpy
all_images = np.array(all_images)
labels = np.array(labels)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(all_images, labels, test_size=0.2, random_state=42)

print(f'Обучающая выборка: {len(X_train)} изображений')
print(f'Тестовая выборка: {len(X_test)} изображений')

Обучающая выборка: 7316 изображений
Тестовая выборка: 1829 изображений


In [20]:
# Импорт модели ResNet-18
model = resnet152(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 101)  # Изменяем последний слой под 101 класс

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [22]:
device

device(type='cuda')

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [24]:
# Создание словаря для кодирования меток
label_to_index = {label: idx for idx, label in enumerate(np.unique(y_train))}
index_to_label = {idx: label for label, idx in label_to_index.items()}

# Преобразование меток в числовые значения
y_train_encoded = [label_to_index[label] for label in y_train]

# Преобразование данных в тензоры
X_train_tensors = [torch.tensor(img, dtype=torch.float32) for img in X_train]
y_train_tensors = torch.tensor(y_train_encoded, dtype=torch.long)

# Создание датасета и загрузчика данных
train_dataset = TensorDataset(torch.stack(X_train_tensors), y_train_tensors)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [25]:
# Функция обучения
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}]  Loss: {loss.item():.6f}')

In [26]:
# Создание словаря для кодирования меток
label_to_index = {label: idx for idx, label in enumerate(np.unique(y_test))}
index_to_label = {idx: label for label, idx in label_to_index.items()}

# Преобразование меток в числовые значения
y_test_encoded = [label_to_index[label] for label in y_test]

# Преобразование данных в тензоры
X_test_tensors = [torch.tensor(img, dtype=torch.float32) for img in X_test]
y_test_tensors = torch.tensor(y_test_encoded, dtype=torch.long)

# Создание датасета и загрузчика данных
test_dataset = TensorDataset(torch.stack(X_test_tensors), y_test_tensors)
test_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [27]:
# Функция тестирования
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.4f})')

In [28]:
# Запуск процесса обучения и тестирования
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[32, 64, 64, 3] to have 3 channels, but got 64 channels instead